In [ ]:
# Importing packages 
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)


In [ ]:
#this is the database we will be extracting from
database = "familyman_live_v4"

#this extracts the latest snapshot from athena
get_snapshot_date = f"SELECT mojap_snapshot_date from {database}.events order by mojap_snapshot_date desc limit 1"
snapshot_date = str(pydb.read_sql_query(get_snapshot_date)['mojap_snapshot_date'].values[0])

#this extracts the February snapshot from athena
#snapshot_date = '2023-02-09'

#this is the athena database we will be storing our tables in
fcsq_database = "fcsq"

#this is the s3 bucket we will be saving data to
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

#change these to the current quarter and year not the quarter being published
latest_quarter = 3
latest_year = 2023

#change these to the current quarter and year being published
pub_quarter = 2
pub_year = 2023

In [ ]:
create_dv_apps_with_app_gender = f"""
SELECT
A.*,
B.ROLE,
B.ROLE_MODEL,
B.PARTY,
B.GENDER
FROM {fcsq_database}.DV_APPS_FINAL A
LEFT JOIN {fcsq_database}.applicants B
ON A.CASE_NUMBER = B.CASE_NUMBER
"""

pydb.create_temp_table(create_dv_apps_with_app_gender,'dv_apps_with_app_gender')

In [ ]:
DV_apps_with_app_gender_count = pydb.read_sql_query("SELECT * from __temp__.DV_apps_with_app_gender")
DV_apps_with_app_gender_count

In [ ]:
create_APP_CASE_GENDER_FLAG_table =f"""
SELECT
  CASE_NUMBER,
  CASE WHEN GENDER = 1 THEN 1 ELSE 0
     END AS MALE,
  CASE WHEN GENDER = 2 THEN 1 ELSE 0
     END AS FEMALE,
  CASE WHEN GENDER <> 1 AND GENDER <> 2 THEN 1 ELSE 0
    END AS UNKNOWN
FROM 
  __temp__.DV_apps_with_app_gender;
"""
pydb.create_temp_table(create_APP_CASE_GENDER_FLAG_table,'APP_CASE_GENDER_FLAG')

In [ ]:
create_APP_CASE_GENDER_COUNT_table =f"""
SELECT
  CASE_NUMBER,
  SUM(MALE) AS MALE,
  SUM(FEMALE) AS FEMALE,
  SUM(UNKNOWN) AS UNKNOWN
FROM
  __temp__.APP_CASE_GENDER_FLAG
GROUP BY
  CASE_NUMBER;
"""
pydb.create_temp_table(create_APP_CASE_GENDER_COUNT_table,'APP_CASE_GENDER_COUNT')

In [ ]:
create_APP_CASE_GENDER_ADD_table =f"""
SELECT
  CASE_NUMBER,
  MALE,
  FEMALE,
  UNKNOWN,
  CASE WHEN MALE > 0 AND FEMALE > 0 THEN 'Both'
       WHEN MALE > 0 THEN 'Male'
       WHEN FEMALE > 0 THEN 'Female'
       ELSE 'Unknown'
    END AS Gender
FROM
  __temp__.APP_CASE_GENDER_COUNT;
"""
pydb.create_temp_table(create_APP_CASE_GENDER_ADD_table,'APP_CASE_GENDER_ADD')

In [ ]:
create_DV_ORDS_WITH_APP_GENDER_table =f"""
SELECT
  A.*,
  B.GENDER
FROM
  {fcsq_database}.DV_ORDS_FINAL AS A
  LEFT JOIN __temp__.APP_CASE_GENDER_ADD AS B
    ON A.CASE_NUMBER = B.CASE_NUMBER;
"""
pydb.create_temp_table(create_DV_ORDS_WITH_APP_GENDER_table,'DV_ORDS_WITH_APP_GENDER')

In [ ]:
create_dv_ords_region = f"""
SELECT
Year, 
Quarter, 
RECEIPT_DATE, 
CASE_NUMBER,  
Event_Court, 
Description, 
TYPE, 
CASE WHEN GENDER = 'Both' THEN 'Unknown'
WHEN GENDER IS NULL THEN 'Unknown' 
ELSE Gender
END AS Gender2,
Case when Year < 2014 then B.Region_Pre2014
Else B.Region
End As Final_Region
FROM
__temp__.DV_ORDS_WITH_APP_GENDER AS A
INNER JOIN {fcsq_database}.COURT_MV_FEB21_DFJ AS B
ON A.EVENT_COURT = B.CODE;
"""
pydb.create_temp_table(create_dv_ords_region,'DV_ORDS_REGION')

In [ ]:
# COURT TRIBUNAL CASE - EVENT COURT 803
DV_ord_reg = pydb.read_sql_query("SELECT * from __temp__.DV_ords_region where final_region = 'COURT TRIB'")
DV_ord_reg

In [ ]:
create_dv_final = f"""
SELECT
YEAR,
QUARTER,
Final_Region,
Description,
Gender2,
COUNT (*) AS COUNT
FROM
  __temp__.DV_ORDS_REGION
WHERE
  YEAR >= 2011
  AND NOT (YEAR = {latest_year} AND QUARTER = {latest_quarter})
GROUP BY
  YEAR,
  QUARTER,
  Final_Region,
  Description,
  Gender2;
  """
pydb.create_temp_table(create_dv_final,'DV_FINAL')

In [ ]:
df = pydb.read_sql_query("SELECT * from __temp__.dv_final")
df.to_csv(path_or_buf = 's3://alpha-family-data/FOIs/domestic_violence_ons.csv',index=False)

In [ ]:
df